## Kalshi Setup

In [4]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
import requests
import datetime

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key

def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')

    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    
def load_key_id_from_file(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()


current_time = datetime.datetime.now()

timestamp = current_time.timestamp()

current_time_milliseconds = int(timestamp * 1000)
timestampt_str = str(current_time_milliseconds)

private_key = load_private_key_from_file('new-elections-key.txt')

method = "GET"
# base_url = 'https://trading-api.kalshi.com'
base_election_url = 'https://api.elections.kalshi.com'
path='/trade-api/v2/portfolio/balance'


msg_string = timestampt_str + method + path

sig = sign_pss_text(private_key, msg_string)

headers = {
        'KALSHI-ACCESS-KEY': load_key_id_from_file('elections-key-id.txt'),
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestampt_str
    }
test_response = requests.get(base_election_url + path, headers=headers)
print("Status Code:", test_response.status_code)

Status Code: 200


In [5]:
import requests
import json

url = "https://api.elections.kalshi.com/trade-api/v2/events/POPVOTE-24"

headers = {"accept": "application/json"}

kalshi_popular_vote_response = requests.get(url, headers=headers)

kalshi_event_data = json.loads(kalshi_popular_vote_response.text)

print(kalshi_event_data)


{'event': {'event_ticker': 'POPVOTE-24', 'series_ticker': 'POPVOTE', 'sub_title': 'In 2024', 'title': 'Who will win the popular vote?', 'mutually_exclusive': True, 'category': 'Politics'}, 'markets': [{'ticker': 'POPVOTE-24-D', 'event_ticker': 'POPVOTE-24', 'market_type': 'binary', 'title': '', 'subtitle': ':: Or another Democrat', 'yes_sub_title': 'Kamala Harris', 'no_sub_title': 'Kamala Harris', 'open_time': '2024-10-07T12:15:00Z', 'close_time': '2025-11-05T15:00:00Z', 'expected_expiration_time': '2025-01-07T15:00:00Z', 'expiration_time': '2025-11-05T15:00:00Z', 'latest_expiration_time': '2025-11-05T15:00:00Z', 'settlement_timer_seconds': 14, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 62, 'yes_ask': 63, 'no_bid': 37, 'no_ask': 38, 'last_price': 63, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 6058024, 'volume_24h': 601112, 'liquidity': 1065448924, 'open_interest': 4013838, 'result': '', 'ca

In [6]:
# Kamala
kalshi_event_data['markets'][0]

{'ticker': 'POPVOTE-24-D',
 'event_ticker': 'POPVOTE-24',
 'market_type': 'binary',
 'title': '',
 'subtitle': ':: Or another Democrat',
 'yes_sub_title': 'Kamala Harris',
 'no_sub_title': 'Kamala Harris',
 'open_time': '2024-10-07T12:15:00Z',
 'close_time': '2025-11-05T15:00:00Z',
 'expected_expiration_time': '2025-01-07T15:00:00Z',
 'expiration_time': '2025-11-05T15:00:00Z',
 'latest_expiration_time': '2025-11-05T15:00:00Z',
 'settlement_timer_seconds': 14,
 'status': 'active',
 'response_price_units': 'usd_cent',
 'notional_value': 100,
 'tick_size': 1,
 'yes_bid': 62,
 'yes_ask': 63,
 'no_bid': 37,
 'no_ask': 38,
 'last_price': 63,
 'previous_yes_bid': 0,
 'previous_yes_ask': 0,
 'previous_price': 0,
 'volume': 6058024,
 'volume_24h': 601112,
 'liquidity': 1065448924,
 'open_interest': 4013838,
 'result': '',
 'can_close_early': True,
 'expiration_value': '',
 'category': '',
 'risk_limit_cents': 0,
 'strike_type': 'custom',
 'custom_strike': {'Election year': '2024', 'Party': 'Dem

In [7]:
kalshi_kamala_yes_ask = kalshi_event_data['markets'][0]['yes_ask'] / 100
kalshi_kamala_no_ask = kalshi_event_data['markets'][0]['no_ask'] / 100


In [8]:
# Trump
kalshi_event_data['markets'][1]

{'ticker': 'POPVOTE-24-R',
 'event_ticker': 'POPVOTE-24',
 'market_type': 'binary',
 'title': '',
 'subtitle': ':: Or another Republican',
 'yes_sub_title': 'Donald Trump',
 'no_sub_title': 'Donald Trump',
 'open_time': '2024-10-07T12:15:00Z',
 'close_time': '2025-11-05T15:00:00Z',
 'expected_expiration_time': '2025-01-07T15:00:00Z',
 'expiration_time': '2025-11-05T15:00:00Z',
 'latest_expiration_time': '2025-11-05T15:00:00Z',
 'settlement_timer_seconds': 14,
 'status': 'active',
 'response_price_units': 'usd_cent',
 'notional_value': 100,
 'tick_size': 1,
 'yes_bid': 37,
 'yes_ask': 38,
 'no_bid': 62,
 'no_ask': 63,
 'last_price': 38,
 'previous_yes_bid': 0,
 'previous_yes_ask': 0,
 'previous_price': 0,
 'volume': 4413721,
 'volume_24h': 270353,
 'liquidity': 1125693829,
 'open_interest': 3001206,
 'result': '',
 'can_close_early': True,
 'expiration_value': '',
 'category': '',
 'risk_limit_cents': 0,
 'strike_type': 'custom',
 'custom_strike': {'Election year': '2024', 'Party': 'Rep

In [9]:
kalshi_trump_yes_ask = kalshi_event_data['markets'][1]['yes_ask'] / 100
kalshi_trump_no_ask = kalshi_event_data['markets'][1]['no_ask'] / 100


## Polymarket Setup

In [10]:
popular_vote_id = '903216'
r = requests.get(f"https://gamma-api.polymarket.com/events/{popular_vote_id}")
polymarket_popular_vote_response = r.json()

polymarket_popular_vote_response

{'id': '903216',
 'ticker': 'presidential-election-popular-vote-winner-2024',
 'slug': 'presidential-election-popular-vote-winner-2024',
 'title': 'Popular Vote Winner 2024',
 'description': 'This is a market on predicting the winner of the popular vote in the 2024 presidential election.',
 'resolutionSource': '',
 'startDate': '2024-01-09T23:37:00Z',
 'creationDate': '2024-01-09T23:53:13.408Z',
 'endDate': '2024-11-04T12:00:00Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/presidential-election-popular-vote-winner-2024-g5iK_jiRrx7q.jpg',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/presidential-election-popular-vote-winner-2024-g5iK_jiRrx7q.jpg',
 'active': True,
 'closed': False,
 'archived': False,
 'new': False,
 'featured': True,
 'restricted': True,
 'liquidity': 120660329.38581,
 'volume': 499442480.0511,
 'openInterest': 0,
 'sortBy': 'price',
 'published_at': '2024-01-09 17:47:51.588+00',
 'updatedBy': '15',
 'createdAt': '2024-01-09T17:47

In [11]:
polymarket_popular_vote_response['markets'][2]

{'id': '253727',
 'question': 'Kamala Harris wins the popular vote?',
 'conditionId': '0x265366ede72d73e137b2b9095a6cdc9be6149290caa295738a95e3d881ad0865',
 'slug': 'will-kamala-harris-win-the-popular-vote-in-the-2024-presidential-election',
 'resolutionSource': '',
 'endDate': '2024-11-05T12:00:00Z',
 'liquidity': '1999837.64441',
 'startDate': '2024-01-09T23:43:00Z',
 'fee': '20000000000000000',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-kamala-harris-win-the-popular-vote-in-the-2024-presidential-election-09127943-2d88-4d4a-9971-2e143ae86bcf.png',
 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-kamala-harris-win-the-popular-vote-in-the-2024-presidential-election-09127943-2d88-4d4a-9971-2e143ae86bcf.png',
 'description': 'This market will resolve to “Yes” if Kamala Harris is the candidate who receives the most votes in the 2024 U.S. presidential election, otherwise this market will resolve to “No.”\n\nIn case of a tie, this market will res

In [12]:
polymarket_kamala_yes, polymarket_kamala_no = map(lambda x: float(x.strip().strip('"')), polymarket_popular_vote_response['markets'][2]['outcomePrices'].strip('[]').split(','))
polymarket_trump_yes, polymarket_trump_no = map(lambda x: float(x.strip().strip('"')), polymarket_popular_vote_response['markets'][3]['outcomePrices'].strip('[]').split(','))


## Arbitrage Calculation


In [13]:
## Arbitrage Calculation

# Print Kalshi variables
print("Kalshi Variables:")
print(f"Kamala Yes: {kalshi_kamala_yes_ask}")
print(f"Kamala No: {kalshi_kamala_no_ask}")
print(f"Trump Yes: {kalshi_trump_yes_ask}")
print(f"Trump No: {kalshi_trump_no_ask}")

# Print Polymarket variables
print("\nPolymarket Variables:")
print(f"Kamala Yes: {polymarket_kamala_yes}")
print(f"Kamala No: {polymarket_kamala_no}")
print(f"Trump Yes: {polymarket_trump_yes}")
print(f"Trump No: {polymarket_trump_no}")

Kalshi Variables:
Kamala Yes: 0.63
Kamala No: 0.38
Trump Yes: 0.38
Trump No: 0.63

Polymarket Variables:
Kamala Yes: 0.5845
Kamala No: 0.4155
Trump Yes: 0.4165
Trump No: 0.5835


In [14]:
polymarket_kamala_yes + kalshi_kamala_no_ask

0.9645

In [15]:
polymarket_trump_no + kalshi_kamala_no_ask

0.9635

In [16]:
cost = polymarket_trump_no + kalshi_kamala_no_ask

In [17]:
print(round((1 - cost) / cost * 100, 2), '% risk-free')

3.79 % risk-free
